По образу практики, попробуйте создать искусственный датасет с лишними столбцами.
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель.
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown. Здесь важно видеть ваш ход мысли.
Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы.
Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Создаем датасет про покупательскую способность:
* age - возраст покупателя
* salary - зарплата покупателя (руб)
* num_of_purchases_per_week - кол-во покупок в неделю (от 0 до 7)
* purchase_probability - вероятность покупки на след.неделе (полнсотью зависит от num_of_purchases_per_week)

In [24]:
# Создаём сэмпл
n_samples = 1000

age = np.random.choice(90, n_samples) + 21
salary = np.random.choice(250000, n_samples) + 15000
num_of_purchases_per_week = np.random.choice(7, n_samples)

purchase_probability = np.round(num_of_purchases_per_week/7, 2) * 100

data = pd.DataFrame({'age': age, 'salary': salary,\
        'num_of_purchases_per_week': num_of_purchases_per_week,\
        'purchase_probability': purchase_probability})
data.head(5)

,age,salary,num_of_purchases_per_week,purchase_probability
0,29,102302,2,29.0
1,65,259013,6,86.0
2,29,23472,2,29.0
3,107,50272,3,43.0
4,31,255767,6,86.0


In [25]:
np.set_printoptions(suppress=True)

Посмотрим значения коэффициентов в нашем датасете для предсказания вероятности покупки:

In [26]:
from sklearn.metrics import mean_absolute_error

X = data[['age', 'salary', 'num_of_purchases_per_week']]
y = data['purchase_probability']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age', 'salary', 'num_of_purchases_per_week']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-0.00031794 -0.00000006 14.27938509]
Bias: 0.035392812154846354
Error: 0.25118816119054127


Ошибка получилась небольшая (у нас значения purchase_probability от 0 до 100, а ошибка - 0.25)
При этом убирать атрибуты, кроме num_of_purchases_per_week не имеет смысла, т.к. их вес очень маленький и их удаление не изменит ошибку в значительной степени:

In [42]:
from sklearn.metrics import mean_absolute_error

X = data[['salary', 'num_of_purchases_per_week']]
y = data['purchase_probability']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['salary', 'num_of_purchases_per_week']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-0.00000006 14.2793693 ]
Bias: 0.014712344657098697
Error: 0.2509477890674743


Как и ожидалось, ошибка почти не изменилась.

Создадим новый признак, который будет коррелировтать с вероятностью покупки на 100% (образован точно так же, но без round и без умножения на 100):

In [37]:
# Создаем новый признак
data['purchase_probability_part'] = data.num_of_purchases_per_week/7
data.head(5)

,age,salary,num_of_purchases_per_week,purchase_probability,purchase_probability_100,purchase_probability_part
0,29,102302,2,29.0,0.285714,0.285714
1,65,259013,6,86.0,0.857143,0.857143
2,29,23472,2,29.0,0.285714,0.285714
3,107,50272,3,43.0,0.428571,0.428571
4,31,255767,6,86.0,0.857143,0.857143


In [38]:
X = data[['purchase_probability_part']]
y = data['purchase_probability']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['purchase_probability_part']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [99.95501383]
Bias: 0.006127558431124669
Error: 0.25098559351385563


Получилась не 100% зависимость (потому что не добавили round).